In [ ]:
# !pip install langchain
# !pip install langchain-community
# !pip install openai
# !pip install unstructured
# !pip install "unstructured[docx]"

In [ ]:
import requests
from bs4 import BeautifulSoup  # HTML 태그를 제거하기 위한 라이브러리
import re  # 정규 표현식을 사용하여 특수 문자를 제거하기 위한 모듈
from langchain.document_loaders import TextLoader  # 텍스트 파일을 로드하기 위한 LangChain의 로더
from langchain.text_splitter import RecursiveCharacterTextSplitter  # 텍스트를 분할하는 모듈
from langchain.embeddings import OpenAIEmbeddings  # OpenAI 임베딩을 생성하는 모듈
from langchain.vectorstores import FAISS  # 벡터 스토어를 생성하는 모듈

class McDonaldsMenu:
    def __init__(self):
        # 메뉴 데이터를 가져오기 위한 API의 URL과 헤더 설정
        self.url = "https://www.mcdonalds.co.kr/kor/menu/listContent.do"
        self.detail_url = "https://www.mcdonalds.co.kr/kor/menu/detail.do"  # 각 메뉴의 상세 정보 URL
        # HTTP 요청을 보낼 때 사용할 헤더
        self.headers = {
            "Content-Type": "application/x-www-form-urlencoded; charset=UTF-8",
            "User-Agent": "Mozilla/5.0"  # 브라우저를 흉내 내기 위해 User-Agent 설정
        }

    def fetch_menu_data(self, sub_category_seq, page_num):
        """
        카테고리와 페이지별로 메뉴 데이터를 가져오는 함수.
        sub_category_seq: 카테고리 번호
        page_num: 페이지 번호
        """
        data = {
            "page": page_num,
            "sub_category_seq": sub_category_seq
        }

        try:
            # POST 요청을 보내고, 응답을 받음
            response = requests.post(self.url, headers=self.headers, data=data)
            response.raise_for_status()  # 4xx, 5xx 오류 처리
            return response.json().get('list', [])  # 'list'가 없으면 빈 리스트 반환
        except requests.exceptions.RequestException as e:
            print(f"API 요청 실패: {e}")
            return []

    def fetch_menu_detail(self, menu_code, sub_category_seq):
        """
        메뉴의 세부 정보를 가져오는 함수.
        menu_code: 각 메뉴의 고유 코드
        sub_category_seq: 단품/세트에 따라 세부 정보를 가져오기 위한 카테고리 값
        """
        data = {"seq": menu_code, "page": 1, "sub_category_seq": sub_category_seq}  # 동적으로 menu_code 및 sub_category_seq 사용

        try:
            # 메뉴의 상세 정보 API 요청
            response = requests.post(self.detail_url, headers=self.headers, data=data)
            response.raise_for_status()

            # HTML 형식으로 응답을 받았을 경우 BeautifulSoup으로 파싱
            soup = BeautifulSoup(response.text, 'html.parser')

            # 메뉴 이름과 설명 가져오기 (None 체크 추가)
            menu_name_tag = soup.find('h2', class_='ko')
            menu_desc_tag = soup.find('div', class_='desc')

            # 특수 기호 ™, ® 제거
            menu_name = menu_name_tag.get_text(strip=True) if menu_name_tag else "메뉴 이름을 찾을 수 없음"
            menu_name = re.sub(r'[™®]', '', menu_name)  # 특수 기호 제거

            menu_desc = menu_desc_tag.get_text(strip=True) if menu_desc_tag else "메뉴 설명을 찾을 수 없음"
            menu_desc = re.sub(r'[™®]', '', menu_desc)  # 특수 기호 제거

            # 판매 시간 패턴 제거
            menu_desc = re.sub(r'\*?판매\s*시간\s*[:：]?\s*[^\*]+', '', menu_desc).strip()

            return {
                "menu_name": menu_name,
                "menu_desc": menu_desc,
            }
        except requests.exceptions.RequestException as e:
            print(f"상세 정보 API 요청 실패: {e}")
            return {}

    def clean_text(self, text):
        """
        HTML 태그와 특수 문자를 제거하는 함수.
        text: 처리할 텍스트
        """
        cleaned_text = BeautifulSoup(text, "html.parser").get_text()
        cleaned_text = re.sub(r'[™®]', '', cleaned_text)  # 특수 기호 제거
        return cleaned_text

    def save_menu_to_file(self, all_menu_items, sub_category_seq):
        """
        모든 메뉴 데이터를 파일에 저장하는 함수.
        all_menu_items: API를 통해 가져온 메뉴 데이터 리스트
        sub_category_seq: 단품/세트 여부를 지정하는 카테고리 값
        """
        with open(f"/content/drive/MyDrive/Aiffel/Project/menu_data_subcategory_{sub_category_seq}.txt", "w", encoding="utf-8") as f:
            for item in all_menu_items:
                kor_name = self.clean_text(item.get('kor_name', ''))
                menu_code = item.get('seq', '')  # 각 메뉴의 코드
                menu_details = self.fetch_menu_detail(menu_code, sub_category_seq)  # 메뉴 상세 정보 가져오기
                f.write(f"{kor_name} - 세부 정보: {menu_details['menu_name']} / {menu_details['menu_desc']}\n")
        print(f"메뉴 데이터를 파일에 저장했습니다. (카테고리 {sub_category_seq})")

# McDonaldsMenu 클래스의 인스턴스 생성
menu = McDonaldsMenu()

# 단품 카테고리(seq=1)와 세트 카테고리(seq=2)에 따라 데이터를 각각 가져옴
for sub_category_seq in [1, 2]:
    all_menu_items = []  # 각 카테고리마다 리스트를 새로 선언하여 초기화
    for page_num in range(1, 6):
        menu_data = menu.fetch_menu_data(sub_category_seq, page_num)
        if not menu_data:
            break
        all_menu_items.extend(menu_data)

    # 단품 또는 세트 데이터를 각각 저장
    menu.save_menu_to_file(all_menu_items, sub_category_seq)
